### 作業目的: 熟練Pytorch Dataset與DataLoader進行資料讀取

本此作業主要會使用[IMDB](http://ai.stanford.edu/~amaas/data/sentiment/)資料集利用Pytorch的Dataset與DataLoader進行
客製化資料讀取。
下載後的資料有分成train與test，因為這份作業目的在讀取資料，所以我們取用train部分來進行練習。
(請同學先行至IMDB下載資料)

### 載入套件

In [2]:
# Import torch and other required modules
import glob
import torch
import os 
import re
import nltk
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_svmlight_file
from nltk.corpus import stopwords

nltk.download('stopwords') #下載stopwords
nltk.download('punkt') #下載word_tokenize需要的corpus

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yumihsu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yumihsu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### 探索資料與資料前處理
在train資料中，有分成pos(positive)與neg(negative)，分別為正評價與負評價，此評價即為label。

In [3]:
# 讀取字典，這份字典為review內所有出現的字詞
with open(os.path.join('E:\\jupyter\\dataset\\aclImdb', 'imdb.vocab'), encoding='utf-8') as f:
    vocab = [line.strip() for line in f.readlines()]
print(vocab[:4])
# 以nltk stopwords移除贅字，過多的贅字無法提供有用的訊息，也可能影響模型的訓練
print(f"vocab length before removing stopwords: {len(vocab)}")
clean_vocab = [ i for i  in vocab if i not in stopwords.words('english')]
print(f"vocab length after removing stopwords: {len(clean_vocab)}")

# 將字典轉換成dictionary
### <your code>###
vocab_dic = dict([(i[1],i[0]) for i in enumerate(clean_vocab)])

['the', 'and', 'a', 'of']
vocab length before removing stopwords: 89527
vocab length after removing stopwords: 89356


In [67]:
# 將資料打包成(x, y)配對，其中x為review的檔案路徑，y為正評(1)或負評(0)
# 這裡將x以檔案路徑代表的原因是讓同學練習不一次將資料全讀取進來，若電腦記憶體夠大(所有資料檔案沒有很大)
# 可以將資料全一次讀取，可以減少在訓練時I/O時間，增加訓練速度
path = 'E:\\jupyter\\dataset\\aclImdb\\train\\'
os.chdir(path)
pos = [(path + 'pos\\' + i, 0) for i in os.listdir('pos')]
neg = [(path + 'neg\\' + i, 1) for i in os.listdir('neg')]
review_pairs = pos + neg
print(review_pairs[:2])
print(f"Total reviews: {len(review_pairs)}")

[('E:\\jupyter\\dataset\\aclImdb\\train\\pos\\0_9.txt', 0), ('E:\\jupyter\\dataset\\aclImdb\\train\\pos\\10000_8.txt', 0)]
Total reviews: 25000


### 建立Dataset與DataLoader讀取資料
這裡我們會需要兩個helper functions，其中一個是讀取資料與清洗資料的函式(load_review)，另外一個是生成詞向量BoW的函式
(generate_bow)

In [82]:
def load_review(review_path):    
    #移除non-alphabet符號、贅字與tokenize
    with open(review_path, 'r', encoding='utf-8') as e:
        review = e.read()
    review = re.sub(r'\W', ' ', review)
    review = nltk.word_tokenize(review)
    return review

In [84]:
def generate_bow(review, vocab_dic):
    bag_vector = np.zeros(len(vocab_dic))
    for word in review:
        if vocab_dic.get(word):
            bag_vector[vocab_dic.get(word)] += 1
    return bag_vector

In [92]:
class dataset(Dataset):
    '''custom dataset to load reviews and labels
    Parameters
    ----------
    data_pairs: list
        directory of all review-label pairs
    vocab: list
        list of vocabularies
    '''
    def __init__(self, data_dirs, vocab):
        self.data_dirs = data_dirs
        self.vocab = vocab

    def __len__(self):
        return len(self.data_dirs)

    def __getitem__(self, idx):
        review_path, label = self.data_dirs[idx]
        review = generate_bow(load_review(review_path), self.vocab)
        return review, label
        

In [94]:
# 建立客製化dataset
custom_dst = dataset(review_pairs, vocab_dic)
custom_dst[10]

(array([0., 2., 1., ..., 0., 0., 0.]), 0)

In [96]:
# 建立dataloader
custom_dataloader = DataLoader(custom_dst)
next(iter(custom_dataloader))

[tensor([[0., 0., 1.,  ..., 0., 0., 0.]], dtype=torch.float64), tensor([0])]